### 상품목록 [] 삭제 후 분석 DB 저장 

In [ ]:
# 먼저 상품목록을 분석 DB에 저장
#

In [6]:
from DB.db import dbcon

In [ ]:
conn = dbcon()
import re

with conn.cursor() as cur:
    cur.execute("SELECT productID, product_name FROM tb_products")
    result = cur.fetchall()

    for i in result:
        productID = i[0]
        name = i[1]
        clean_name = re.sub("\[.*?\]", "", name)

    #         cur.execute("INSERT INTO tb_분석AI테이블명 (productID, clean_name) VALUES (%s,%s)",(productID, clean_name))
    # conn.commit()

NEW컬러! 카스테라 워싱 옥수수솜 간절기/사계절 차렵이불세트
비침없는 도톰 레이스/쉬폰커튼(나비주름/핀형/봉집)


### 리뷰 내용 중 별점 선택 시 자동으로 들어가는 텍스트 삭제하고 분석 DB 저장

In [ ]:
# 리뷰 내용 중 너무 짧은 내용은 삭제하고 분석 테이블에 리뷰 내용 추가하도록
# 별점-> 최고예요, 마음에 들어요, 보통이에요, 별로예요, 매우 아쉬워요 자동으로 들어가는 텍스트임

In [ ]:
# 제외할 단독 리뷰 문구
exclude_texts = ["최고예요", "마음에 들어요", "보통이에요", "별로예요", "매우 아쉬워요"]

conn = dbcon()

excluded_reviews = []  # 제외된 리뷰 저장
inserted_reviews = []  # 삽입될 리뷰 저장

with conn.cursor() as cur:
    # 원본 리뷰 불러오기 
    cur.execute("SELECT reviewID, comment FROM tb_reviews")
    result = cur.fetchall()

    for row in result:
        review_id = row[0]
        review_text = row[1]

        # 제외 문구: 정확히 일치할 때만 제외
        if review_text in exclude_texts:
            excluded_reviews.append((review_id, review_text))
            continue
        
        inserted_reviews.append((review_id, review_text))

        # cur.execute("INSERT INTO tb_analyzer (reviewID, comment) VALUES (%s,%s)",(review_id, review_text))
    # conn.commit()
# 커밋 전 확인
# print("제외된 리뷰:")
# for rid, rtext in excluded_reviews:
#     print(f"- [{rid}] {rtext}")

print("\n삽입될 리뷰:")
for rid, rtext in inserted_reviews:
    print(f"- [{rid}] {rtext}")



삽입될 리뷰:
- [53305893] 제품을 직접 사용해 보니 품질이 기대 이상으로 훌륭합니다. 마감이 깔끔하고 내구성도 좋아 오래 사용할 수 있을 것 같아요. 사용감도 편리하고 성능도 만족스러워서 재구매 의사 100%입니다. 주변에도 자신 있게 추천할 만한 제품이에요.
- [53305745] 색감도 너무 예쁘고 사이즈 적당하고 푹신푹신합니다
- [53291001] 땅에 닿는게 싫어서 쪼금 더 짧게 주문했는데 
길이 너무 맘에 들어요 ㅎㅎㅎ 원하던대로 ㅎㅎ
예쁩니다!!!! 커튼 재질도 좋고 가볍고 너무 좋아여!!!!
- [53290715] 고르고 고르고 또 생각하고 구매했어요.
그런만큼 정말 만족스러워요.
이걸 속지로 쓰고, 겉지를 따로 구매하려했는데
넘 예뻐서 겉지를 안하기로 했네요.
원단도 나비주름도 넘 고급스러워요 ❤️ 
- [53308210] 지금까지 잘쓰고 있네요 포근하고 좋아요~
- [53308125] 선물햇는데 받는 사람이 너무 죠아햇어요
- [53306498] 보들보들하고 세탁기에 돌리기도 편해서 애용 중입니다
- [53306432] 너무예뻐요 ,,, 포근포근해서 잠도 엄청 잘 오고 사계절용이라 에어컨 틀고 여름에 덮어도 괜찮습니다 ~~! 배송도 빨라서 택배 휴무 전에 바로 받을 수 있었어요 ㅎㅎ 
- [53294962] 결제 후에 따로 언제 배송오는지 사전에 연락이 와서 좋았어요. 지그재그로 접혀 배송되었는데 스팀다리미로 슥슥 밀어주니 주름도 많이 사라졌습니다. 혼자서도 레일 달기 쉽게 안내서도 함께 동봉되서 좋았구요~
햇빛은 은은하게 들어오면서 저녁에는 가려줘 프라이버시는 보장되네요
- [53316092] 이불감촉이 부들부들하고 살에 닿는느낌이 좋으네요
- [53313553] 기대한 것보다 더 부드럽고 좋아요 부드러우면서도 냉감이 시원하게 느껴지니 이보다 더 좋을 순 없네요 감사합니다 아이들 하나씩 장만해 줬는데 모두 만족했어요 
- [53313020] 완전 촉감 부드러워요!!
사계절 사용할수 있긴 하지만 한여름에는 좀 더워요.
저희집 사람

## AI 분석

In [ ]:
# - OpenAI API를 활용해 키워드, 카테고리, 감성 분석 기능 구현.
# - 분석 결과를 DB에 저장하는 함수 작성.

### 리뷰 키워드 추출

In [26]:
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
import time

load_dotenv()

True

In [39]:
prompt = ChatPromptTemplate.from_template(
    """
    당신은 고객 리뷰 데이터를 분석하는 전문가입니다.
    아래 리뷰 텍스트에서 **제품 관련 핵심 명사 키워드 3~5개**를 추출해주세요.

    규칙:
    - 명사만 추출합니다. (예: "색감", "사이즈", "내구성", "마감", "쿠션감")
    - 감정 표현이나 불필요한 단어("좋아요", "최고", "만족")는 제외합니다.
    - 제품의 특징, 장점, 단점, 사용 경험과 관련된 단어를 중심으로 선택합니다.
    - JSON 형식으로 출력합니다.

    리뷰 ID: {review_id}
    리뷰:
    "{review_text}"

    출력 예시:
    {{
      "review_id": {review_id},
      "keywords": ["색감", "사이즈", "내구성"]
    }}
    """
)

llm = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | llm | JsonOutputParser()

all_results = []

# 슬라이싱으로 10개씩 처리
batch_size = 10
for i in range(0, len(inserted_reviews), batch_size):
    batch = inserted_reviews[i : i+batch_size]  # i부터 i+10까지 자르기
    inputs = [{"review_id": rid, "review_text": rtext} for rid, rtext in batch]
    time.sleep(2)
    # LLM batch 호출
    print(f"{i//batch_size + 1}번째 실행 중...")
    results = chain.batch(inputs)
    all_results.extend(results)

# 결과 확인
for r in all_results:
    print(r)

1번째 실행 중...


Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=7707b329-481a-43ed-8d39-b7a5f327f3f8,id=7707b329-481a-43ed-8d39-b7a5f327f3f8; trace=b3ab56db-318e-4a5f-987d-bf3659840553,id=b3ab56db-318e-4a5f-987d-bf3659840553; trace=d22d32e8-572a-40c8-b7f1-5837dd14e3ab,id=d22d32e8-572a-40c8-b7f1-5837dd14e3ab; trace=8a67665b-5bdf-46d0-ad9b-d7800f82537c,id=8a67665b-5bdf-46d0-ad9b-d7800f82537c; trace=68468659-a8f8-4c56-896b-3fe53e19632e,id=68468659-a8f8-4c56-896b-3fe53e19632e; trace=358262d9-c9ea-4ebc-a8ab-40e444ae1c6b,id=358262d9-c9ea-4ebc-a8ab-40e444ae1c6b; trace=f21008f6-f258-442e-bdd4-14e41c7e27a8,id=f21008f6-f258-442e-bdd4-14e41c7e27a8; trace=8aa34a47-3311-41d6-969f-a40d8830d9ae

2번째 실행 중...


Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=ffc50b00-d299-4b76-8c57-3ba221b71802,id=ffc50b00-d299-4b76-8c57-3ba221b71802; trace=d2b131ce-a709-4547-93e1-d45b696a0e68,id=d2b131ce-a709-4547-93e1-d45b696a0e68; trace=bdc24f38-7375-4930-962a-875a222ca3f2,id=bdc24f38-7375-4930-962a-875a222ca3f2; trace=406e3253-fcb1-4ef9-9655-e082db403596,id=406e3253-fcb1-4ef9-9655-e082db403596; trace=b00532cc-cffc-473d-8bbf-be01acbf86df,id=b00532cc-cffc-473d-8bbf-be01acbf86df; trace=d1483d7d-41e9-4d38-8232-cb8d505c3f10,id=d1483d7d-41e9-4d38-8232-cb8d505c3f10; trace=9961c227-e74c-407d-b562-810615ca36b9,id=9961c227-e74c-407d-b562-810615ca36b9; trace=75a9be4f-8542-4fa6-bac6-feafe687a9d9

3번째 실행 중...


...

Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=25eef1e1-e198-4e2b-9de2-d75e79e59ac3,id=25eef1e1-e198-4e2b-9de2-d75e79e59ac3; trace=49221551-9964-4a85-90c0-42adeabda203,id=49221551-9964-4a85-90c0-42adeabda203; trace=e5ac6c19-88e8-49d4-8cbd-a45b4757dce5,id=e5ac6c19-88e8-49d4-8cbd-a45b4757dce5; trace=c42aefe5-7ef1-4d9e-811f-12d653d0a4ab,id=c42aefe5-7ef1-4d9e-811f-12d653d0a4ab; trace=0466b1b9-806d-4dea-9960-34d0adbd3058,id=0466b1b9-806d-4dea-9960-34d0adbd3058; trace=51d7d1b4-2266-49b6-a859-700437cbe8d3,id=51d7d1b4-2266-49b6-a859-700437cbe8d3; trace=c1bd2db9-2772-4023-afa9-011de0d70053,id=c1bd2db9-2772-4023-afa9-011de0d70053; trace=9c4841f3-84a5-4c01-81f6-6e25a0535748

7번째 실행 중...


Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=3e4b19a4-fa5c-4dad-bde3-581c98827c86,id=3e4b19a4-fa5c-4dad-bde3-581c98827c86; trace=3a128232-b3f4-455b-9156-8aaf8c43195a,id=3a128232-b3f4-455b-9156-8aaf8c43195a; trace=7a0fc620-e074-4b4b-8380-0bd36b47a87f,id=7a0fc620-e074-4b4b-8380-0bd36b47a87f; trace=e4c021e6-b909-4ef6-a1ed-7d3a43a7279b,id=e4c021e6-b909-4ef6-a1ed-7d3a43a7279b; trace=87fe50f3-7eef-40c8-8dfa-d3dc2ed5506d,id=87fe50f3-7eef-40c8-8dfa-d3dc2ed5506d; trace=9b306026-62b7-4010-87bd-7b636fd3a737,id=9b306026-62b7-4010-87bd-7b636fd3a737; trace=5e4e9716-c0dd-42bd-8209-c576f9760397,id=5e4e9716-c0dd-42bd-8209-c576f9760397; trace=3e4b19a4-fa5c-4dad-bde3-581c98827c86

{'review_id': 53305893, 'keywords': ['품질', '마감', '내구성', '사용감', '성능']}
{'review_id': 53305745, 'keywords': ['색감', '사이즈', '쿠션감']}
{'review_id': 53291001, 'keywords': ['길이', '커튼', '재질', '가벼움']}
{'review_id': 53290715, 'keywords': ['원단', '나비주름', '고급스러움']}
{'review_id': 53308210, 'keywords': ['포근함']}
{'review_id': 53308125, 'keywords': []}
{'review_id': 53306498, 'keywords': ['보들보들함', '세탁기', '편리함']}
{'review_id': 53306432, 'keywords': ['포근함', '사계절용', '배송']}
{'review_id': 53294962, 'keywords': ['배송', '주름', '안내서', '프라이버시']}
{'review_id': 53316092, 'keywords': ['이불', '감촉', '느낌']}
{'review_id': 53313553, 'keywords': ['부드러움', '냉감', '아이들']}
{'review_id': 53313020, 'keywords': ['촉감', '사계절', '여름', '이불', '편']}
{'review_id': 53312735, 'keywords': ['색감', '보온', '간절기', '이불']}
{'review_id': 53308947, 'keywords': ['원단']}
{'review_id': 53315265, 'keywords': ['커텐', '사이즈', '천', '나비주름']}
{'review_id': 53312244, 'keywords': ['채광', '고급스러움', '커텐']}
{'review_id': 53310616, 'keywords': ['커튼', '천', '레일', '주름', '배송'

Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=5e4e9716-c0dd-42bd-8209-c576f9760397,id=85b96560-061e-4b80-bb3e-55360a49bc94; trace=9b306026-62b7-4010-87bd-7b636fd3a737,id=6d331c50-4137-4adc-9042-406441670fa0; trace=7a0fc620-e074-4b4b-8380-0bd36b47a87f,id=6e79317e-f306-41f1-be90-b186317c8149; trace=e4c021e6-b909-4ef6-a1ed-7d3a43a7279b,id=7d17b3ca-5fc7-4eba-b55a-bea50a282bb9; trace=3e4b19a4-fa5c-4dad-bde3-581c98827c86,id=5f5e686c-d818-4452-b682-294198279e32; trace=3a128232-b3f4-455b-9156-8aaf8c43195a,id=18dbf24f-9e55-46ea-a698-5c9e120fcf93; trace=87fe50f3-7eef-40c8-8dfa-d3dc2ed5506d,id=a8ca611c-bb0d-4efa-8221-2e22bda23388; trace=3e4b19a4-fa5c-4dad-bde3-581c98827c86

In [42]:
all_results

[{'review_id': 53305893, 'keywords': ['품질', '마감', '내구성', '사용감', '성능']},
 {'review_id': 53305745, 'keywords': ['색감', '사이즈', '쿠션감']},
 {'review_id': 53291001, 'keywords': ['길이', '커튼', '재질', '가벼움']},
 {'review_id': 53290715, 'keywords': ['원단', '나비주름', '고급스러움']},
 {'review_id': 53308210, 'keywords': ['포근함']},
 {'review_id': 53308125, 'keywords': []},
 {'review_id': 53306498, 'keywords': ['보들보들함', '세탁기', '편리함']},
 {'review_id': 53306432, 'keywords': ['포근함', '사계절용', '배송']},
 {'review_id': 53294962, 'keywords': ['배송', '주름', '안내서', '프라이버시']},
 {'review_id': 53316092, 'keywords': ['이불', '감촉', '느낌']},
 {'review_id': 53313553, 'keywords': ['부드러움', '냉감', '아이들']},
 {'review_id': 53313020, 'keywords': ['촉감', '사계절', '여름', '이불', '편']},
 {'review_id': 53312735, 'keywords': ['색감', '보온', '간절기', '이불']},
 {'review_id': 53308947, 'keywords': ['원단']},
 {'review_id': 53315265, 'keywords': ['커텐', '사이즈', '천', '나비주름']},
 {'review_id': 53312244, 'keywords': ['채광', '고급스러움', '커텐']},
 {'review_id': 53310616, 'keywor

### 리뷰 카테고리 분류

In [ ]:
prompt = PromptTemplate.from_template(
    """Classify the given user question into only one of the following categories:
    배송: 
    사용감: 이용자들이 사용한 후 경험으로 "부드럽다", "촉촉하다", "자극적이다" 이런 리뷰 위주로 분류해줘.
    사이즈: 
    디자인: 
    품질: 

    If there are multiple categories mixed in the content, return multiple categories. 매치되는 카테고리가 없으면 None으로 출력해줘.
    Return the result in JSON format. 주석 없이, 모든 필드는 문자열로 작성하십시오.
    Content: {Content}
    category: 배송, 사용감, 사이즈, 디자인, 품질, None"""
)

llm = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | llm | JsonOutputParser()

